In [96]:
import os
import pandas as pd
import numpy as np 
import regex as re
from collections import defaultdict

In [4]:
os.chdir("../users")

In [14]:
user_summary = pd.read_csv("../user-summary.csv")

In [335]:
user_summary.user_id.sample(1).values[0]

28849

In [284]:
sites = pd.read_csv("../sites.csv")

ADD STUFF TO GET RID OF THE USERS WE DON'T HAVE DATA FOR

In [322]:
def block_randomization(user_summary, sites, n):
    '''
    Combined user look up table with 
    sites table to get all possible
    combinations of group/site pairs
    
    Performs block randomization on 
    group/site combos, to get n examples
    from each 
    
    Returns a dataframe with the info
    about the sampled simulated
    users
    
    ARGS:
    user_summary = user-summary.csv in 
                   dataframe form 
    
    sites = .csv file created by the 
            output of the
            create_site_look_up()
            function 
    
    n = number of examples to sample from 
        each group/site combo
    '''
    users_df = user_summary.copy()
    users_df.user_id = users_df.user_id.apply(lambda s: f'user_{s}')
    users_df['key'] = 1
    users_df = users_df[['user_id', 'group', 'key']].values
    users_df = pd.DataFrame(users_df)
    users_df.columns = ['user', 'group', 'key']
    
    sites['key'] = 1
    
    all_sites = users_df.merge(sites, on = 'key', how = 'outer')
    
    sample_df = all_sites.groupby(['suffix', 'group']).sample(n, random_state = 42)
    
    return sample_df
    

In [323]:
t = block_randomization(user_summary, sites, 4)

In [332]:
sites

,suffix,site,key
0,0,https://www.youtube.com/,1
1,1,https://www.yahoo.com/,1
2,2,https://www.espn.com/,1
3,3,https://weather.com/,1
4,4,https://www.accuweather.com/,1
5,5,https://www.fandom.com/,1
6,6,https://www.twitch.tv/,1
7,7,https://definition.org/,1
8,8,https://www.aol.com/,1
9,9,https://www.imdb.com/,1


Development code this is junk do not run

In [314]:
t = get_folders(user_summary, 2)

In [240]:
sampled_users = user_summary.groupby('group').sample(n, random_state = 42)
# ids = groups[]
# ids = [f'users_{id_}' for id_ in ids]

In [250]:
sites = pd.read_csv("../sites.csv")

In [271]:
users = sampled_users[['user_id', 'group']].values
users = [[f'user_{user[0]}', user[1]] for user in users]
test = pd.DataFrame(users)
test.columns = ['user', 'group']

In [272]:
users = user_summary.user_id.values

In [273]:
test['key'] = 1
sites['key'] = 1

In [278]:
user_summary['key'] = 1

In [279]:
all_sites = user_summary.merge(sites, on = 'key', how = 'outer')

In [283]:
all_sites.groupby(['suffix', 'group']).sample(3)

,user_id,group,vm_id,vpn_location,vpn_server,proportion,key,suffix,site
219840,6046,abortion - seeking,6,Miami,2,20,1,0,https://www.youtube.com/
210080,5802,abortion - seeking,6,Cincinnati,13,75,1,0,https://www.youtube.com/
204040,5651,abortion - seeking,6,Denver,13,84,1,0,https://www.youtube.com/
722040,18601,addiction - alcohol,7,Charlotte,34,63,1,0,https://www.youtube.com/
738840,19021,addiction - alcohol,7,San Jose,44,65,1,0,https://www.youtube.com/
...,...,...,...,...,...,...,...,...,...
409319,10782,survivor - domestic abuse,11,Chicago,25,6,1,39,http://www.allscrabblewords.com/
411119,10827,survivor - domestic abuse,11,San Jose,8,20,1,39,http://www.allscrabblewords.com/
927479,23736,survivor - sexual assualt,12,Chicago,33,1,1,39,http://www.allscrabblewords.com/
921079,23576,survivor - sexual assualt,12,Phoenix,8,33,1,39,http://www.allscrabblewords.com/


In [68]:
def get_user_folders_OLD(working_dir = '.'):
    '''
    Finds all the files in a directory 
    that contain data for simulated users
    default is the current directory 
    
    Returns list of files 
    
    ARGS:
    working_dir = directory to look for files in 
                  default behavior is the current 
                  working directory
    '''
    folders = list(os.listdir(working_dir))
    folders = [f for f in folders if (re.match('(user_\d{3,5})', f) is not None)]
    return folders

In [231]:
sample_folders(user_summary, 1)

['users_5788',
 'users_5851',
 'users_5591',
 'users_5934',
 'users_18837',
 'users_19017',
 'users_18946',
 'users_19030',
 'users_19678',
 'users_19658',
 'users_19916',
 'users_19651',
 'users_19211',
 'users_19448',
 'users_19462',
 'users_19337',
 'users_15471',
 'users_15529',
 'users_15439',
 'users_15398',
 'users_14771',
 'users_14845',
 'users_14897',
 'users_14791',
 'users_24280',
 'users_24122',
 'users_24174',
 'users_24497',
 'users_23516',
 'users_23155',
 'users_23343',
 'users_23482',
 'users_23011',
 'users_22995',
 'users_23015',
 'users_22588',
 'users_18383',
 'users_18248',
 'users_18165',
 'users_18257',
 'users_17758',
 'users_17823',
 'users_17617',
 'users_17766',
 'users_21758',
 'users_21985',
 'users_22042',
 'users_21779',
 'users_22325',
 'users_22111',
 'users_22149',
 'users_22061',
 'users_12549',
 'users_12186',
 'users_12060',
 'users_12448',
 'users_11606',
 'users_11586',
 'users_11840',
 'users_11630',
 'users_11292',
 'users_11166',
 'users_1154

In [ ]:
def folders_to_groups_OLD(folders):
    '''
    Takes in a list of user folders
    associated them with a sensitive group 
    Returns a dataframe 
    
    ARGS:
    folders = list of folders in the form user_\d{3-4}
    '''
    folders_to_users = defaultdict()
    for folder in folders:
        group = user_summary[user_summary.user_id == int(folder[5:])].group.values[0]
        folders_to_users[folder] = group
        
    folder_groups_df = pd.DataFrame.from_dict(folders_to_users, orient = 'index').reset_index()
    folder_groups_df.columns = ['folder', 'group']

In [ ]:
def sample_folders_OLD(folders, user_summary, n):
    '''
    Generates a list of unique groups from 
    user summary table
    
    Samples full list of folders evenly 
    among groups 
    
    Returns a list of folders
    
    ARGS:
    folders = list of folders to sample from 
    user_summary = dataframe with a list of all simulated users
    n = number of folders to sample from each group
    '''
    groups = user_summary.group.unique()
    
    samples = []
    for group in groups: 
        groups_folders = folder_groups_df[folder_groups_df.group == group]
        sample = groups_folders.sample(n, random_state = 42)
        samples.extend(list(sample.folder.values))
    
    return samples
    

In [69]:
folders = get_user_folders()

In [214]:
folders = ['user_550', 'user_551', 'user_552', 'user_30548', 'user_1053', 'user_21958', 'user_1689', 'user_30547', 'user_30546']

In [217]:
groups = user_summary.groupby('group').sample(4)

In [219]:
ids = groups.user_id

In [222]:
test = [f'users_{id_}' for id_ in ids]

In [182]:
folders_to_users = defaultdict()
for folder in folders:
    group = user_summary[user_summary.user_id == int(folder[5:])].group.values[0]
    folders_to_users[folder] = group

In [183]:
folder_groups_df = pd.DataFrame.from_dict(folders_to_users, orient = 'index').reset_index()
folder_groups_df.columns = ['folder', 'group']

In [184]:
groups = user_summary.group.unique()

In [196]:
n = 1

samples = []
for group in ['near miss - deals coupons', 'race - black']: 
    groups_folders = folder_groups_df[folder_groups_df.group == groups[-1]]
    sample = groups_folders.sample(2, random_state = 42)
    samples.extend(list(sample.folder.values))

In [197]:
samples

['user_30548', 'user_30547', 'user_30548', 'user_30547']

In [195]:
list(sample.folder.values)

['user_30548', 'user_30547']

In [204]:
sampled_folders = ['user_1000']

In [205]:
os.getcwd()

'/Users/haleyjohnson/Desktop/users'

In [212]:
os.listdir(f'{sampled_folders[0]}/screenshot_0/data/screenshots/parts')

['7685760969266350-f983c4d08608a843e379a2b9ee033212-_19-part-16-10533.png',
 '6264624114603628-69e66981fb6f154068fb205e915bfbe6-_2-part-7-4828.png',
 '5547622385311732-869aa6d4178b4cc8a4cd0af3276979c5-_22-part-25-18002.png',
 '283131688007233-f8efc08be8aa3e9c2fe11cbeec2e03f6-_36-part-5-3074.png',
 '552545419390414-6f4f13c21bb217801ab4b37c367f97aa-_25-part-14-9075.png',
 '6435949136312956-458ddf355ab77631d3eb3dd84a0351b4-_16-part-10-13262.png',
 '2416843165171923-283e29c34c5097fbdf1356d886af43fd-_12-part-7-4528.png',
 '3522410619630957-84b4a4653098f5c028d38245a4278070-_13-part-4-2766.png',
 '5547622385311732-869aa6d4178b4cc8a4cd0af3276979c5-_22-part-6-4332.png',
 '3717897767502739-32543af074876960acc4e7f3ad4e3ac7-_30-part-8-6068.png',
 '5547622385311732-869aa6d4178b4cc8a4cd0af3276979c5-_22-part-28-19928.png',
 '3822960338427677-7b1ef53768d81ce9cb762217eea6d653-_8-part-6-3856.png',
 '2299500491318870-dc954d520aadb08c84108ca1486e9462-_35-part-0-679.png',
 '2319249190312226-20c6e32d611c5ba